# Sistemas de ecuaciones no lineales

Se considera ahora el problema de encontrar el valor de varias variables a partir de un sistema de ecuaciones. En particular, dado un sistema de $N$ ecuaciones, se buscará el valor de $N$ incógnitas. Obsérvese, por un lado, que en caso de tener un número de incóginas ($M$) mayor que el número de ecuaciones, se tendrá que bajo ciertas condiciones (tal y como se ha estudiado en la asignatura *Cálculo II*) se podrá asegurar que $N$ variables pueden escribirse como funciones del resto ($M-N$), pudiendo entonces considerar al resto de las variables ($M-N$) como parámetros de las ecuaciones. En lo sucesivo, se considerará exclusivamente el caso de un sistema de $N$ ecuaciones con $N$ incógnitas. 

Un caso particular es, desde luego, cuando todas las ecuaciones hacen aparecer linealmente a las variables (esto es, tenemos un sistema de $N$ ecuaciones lineales). En este caso, ya se han estudiado métodos (tanto directos como iterativos) para resolver el sistema de ecuaciones. Lo que se va a estudiar a continuación es cómo resolver numéricamente el caso general, donde las ecuaciones hacen aparecer de forma cualquiera a las variables (aunque sí se deberá asumir, como se detallará, una cierta regularidad en las funciones que aparecen).

De modo general, se considera entonces el problema de encontrar los valores $x_1$, $x_2$, $\ldots$ y $x_N$ que satisfacen el siguiente sistema de ($N$) ecuaciones

$$
\begin{array}{cc} 
F_1(x_1, x_2, \ldots, x_N) & = 0, \\
F_2(x_1, x_2, \ldots, x_N) & = 0, \\
\vdots & \\
F_N(x_1, x_2, \ldots, x_N) & = 0,
\end{array}
$$

Podemos reescribir el problema de forma vectorial, a fin de tener una expresión más compacta de los métodos. Se introduce para ello una función vectorial $\mathbf F: D \subset \mathbb R^N \rightarrow \mathbb R^N$. De esta forma, resolver el sistema de ecuaciones no lineales consistirá en encontrar $\mathbf x_* \in D$ tal que $\mathbf F(\mathbf x_*) = \mathbf 0$.

## Método de Newton

Una de las ventajas del método de Newton-Raphson, además de su velocidad de convergencia, es que puede extenderse de forma más o menos inmediata para resolver sistemas de ecuaciones no lineales. Suponiendo $\mathbf x_n$ una cierta aproximación de $\mathbf x_*$, es posible sustituir $\mathbf F(\mathbf x)$ por su desarrollo de Taylor de grado uno en torno a $\mathbf x_n$, esto es,

$$\mathbf P_n(\mathbf x) = \mathbf F(\mathbf x_n) + (\mathbf x -\mathbf x_n) D\mathbf F(\mathbf x_n),$$

donde la *matriz jacobiana* de $\mathbf F$ en $\mathbf x_n$ se define como:

$$
D\mathbf F(\mathbf x_n) = 
\left(\begin{array}{ccc} 
\displaystyle\frac{\partial F_1}{\partial x_1}(\mathbf x_n) & \ldots & \displaystyle\frac{\partial F_1}{\partial x_N}(\mathbf x_n)\\
\vdots & & \vdots \\
\displaystyle\frac{\partial F_N}{\partial x_1}(\mathbf x_n) & \ldots & \displaystyle\frac{\partial F_N}{\partial x_N}(\mathbf x_n)
\end{array}\right).
$$ 

De esta forma, siempre que $D\mathbf F$ sea invertible en el entorno de $\mathbf x_*$, se puede calcular $\mathbf x_{n+1}$ resolviendo 
$$ \mathbf P_n(\mathbf x_{n+1}) = \mathbf 0  $$
tal y como se hacía en el método de Newton-Raphson (cuando solamente se tenía una ecuación y el polinomio de Taylor era una función escalar de una sola variable). La ecuación vectorial anterior representa entonces un sistema de ecuaciones lineales para las compoenntes del vector $\mathbf x_{n+1}$:

$$\mathbf F(\mathbf x_n) + (\mathbf x_{n+1} -\mathbf x_n) D\mathbf F(\mathbf x_n) = \mathbf 0.$$

Reorganizando términos para reducir el número de operaciones realizadas, se calcula $\mathbf x_{n+1}$ como:

$$\mathbf x_{n+1} = \mathbf x_n + \boldsymbol \Delta x_n,$$

donde $\boldsymbol \Delta x_n$ es la solución del sistema lineal:

$$D\mathbf F(\mathbf x_n) \boldsymbol \Delta x_n = - \mathbf F(\mathbf x_n).$$

A continuación se muestra un ejemplo de implementación del algoritmo en Python 3.8.10.

In [2]:
import numpy as np

def newton_sist(f, df, x, tol, nitmax):

    # Inicializacion
    x0 = x

    for nit in range(nitmax):

        # Calculo de f(x_n)
        f0 = f(x0)

        # Calculo de Jf(x_n)
        jacob = df(x0)

        # Calculo de dxn
        dx = np.linalg.solve(jacob, f0)

        # Calculo de xn+1
        x = x0 - dx

        # Criterio de convergencia
        norma_dx = np.linalg.norm(dx, 2)

        print('Iteracion %d, x* = [%0.9f, %0.9f] y |x-xn| = %0.9e' % (nit+1, x[0], x[1], norma_dx))

        if norma_dx < tol:
            break # Convergencia alcanzada
        else:
            x0 = x # Actualizacion para nueva iteracion

    return x, norma_dx, nit

<ul>
    <li>
        <i>Ejemplo 4</i>:
    </li>
</ul>

Se propone utilizar el anterior código para calcular los puntos de corte de la circunferencia $x^2 + y^2 = 2$ y la recta $x = y$. De otro modo, se buscan $x$ e $y$ tales que se verifique el sistema de ecuaciones (no lineales):

$$ x^2 + y^2 = 2$$
$$ x = y$$

Para escribir el sistema de la forma (vectorial) $\mathbf F (\mathbf x) = \mathbf 0$, se tomará $\mathbf x = (x, y)$ así como $\mathbf F = (F_1, F_2)$. De esta forma, se define

$$
\begin{array}{rcl} 
F_1(x_1,x_2) & = & x^2 + y^2 - 2,\\
F_2(x_1,x_2) & = & x - y.
\end{array}
$$

Así, se obtiene la siguiente matriz jacobiana:

$$
D\mathbf F(x, y) = 
\left(\begin{array}{cc} 
\displaystyle\frac{\partial F_1}{\partial x}(x, y) & \displaystyle\frac{\partial F_1}{\partial y}(x, y)\\
\\
\displaystyle\frac{\partial F_2}{\partial x}(x, y) & \displaystyle\frac{\partial F_2}{\partial y}(x, y)
\end{array}\right) =
\left(\begin{array}{cc} 
2x & 2y\\
1 & -1
\end{array}\right).
$$

El siguiente código resuelve el sistema de ecuaciones no lineales utilizando la implementación previa del método de Newton. Como se ve, el comportamiento del método en lo que tiene que ver con la velocidad de convergencia es muy similar al que presentaba en el caso escalar (esto es, para una única ecuación con una incógnita). A continuación se verá un resultado que muestra, en efecto, la generalización de las propiedades de convergencia local del método.

In [3]:
import numpy as np

# Definimos la funcion
def f(x):

    fval = np.array([x[0]**2 + x[1]**2 - 2,
                     x[0] - x[1]])

    return fval

# Definimos de la matriz jacobiana
def df(x):

    jac = np.array([[2*x[0], 2*x[1]],
                    [1.0,    -1.0]])

    return jac

x0     = np.array([0.6, 1.3]) # Iterante inicial
tol    = 1e-8 # Tolerancia
nitmax = 40   # Numero maximo de iteraciones

x, res, nit = newton_sist(f, df, x0, tol, nitmax)

Iteracion 1, x* = [1.065789474, 1.065789474] y |x-xn| = 5.213582304e-01
Iteracion 2, x* = [1.002030539, 1.002030539] y |x-xn| = 9.016874971e-02
Iteracion 3, x* = [1.000002057, 1.000002057] y |x-xn| = 2.868706676e-03
Iteracion 4, x* = [1.000000000, 1.000000000] y |x-xn| = 2.909553861e-06
Iteracion 5, x* = [1.000000000, 1.000000000] y |x-xn| = 2.992910245e-12


### Convergencia del método de Newton

Cabe esperar que las propiedades de convergencia local del método sean similares a las del método de Newton-Raphson para ecuaciones no lineales escalares. Recordemos que en el caso escalar para asegurar la convergencia cuadrática del algoritmo era necesario que la raíz fuese simple, lo que implicaba $f'(x_*) \neq 0$. Por tanto, para el método de Newton para sistemas de ecuaciones no lineales parece natural que dicha condición se transforme en una hipótesis de regularidad sobre la matriz jacobiana $D\mathbf F(\mathbf x_*)$. En este caso, requeriremos que la matriz Jacobiana sea invertible.

Obsérvese que el problema considerado en el ejemplo 4 verifica en efecto las hipótesis de este teorema y la convergencia observada refleja, en efecto, la convergencia cuadrática que este teorema asegura. Se va a considerar ahora un caso en que falla la condición sobre la invertibilidad de la matriz jacobiana (que correspondería en el caso escalar a que la derivada de $f$ se cancele en la raíz donde, como se pedía comprobar, la convergencia del método de Newton-Raphson era mucho más lenta).   

<ul>
    <li>
        <i>Ejemplo 5</i>:
    </li>
</ul>

Se va a usar el método de Newton para encontrar la (única) solución del siguiente sistema de ecuaciones no lineales:

$$ \exp(x^2 + y^2) - 1 = 0,$$
$$ \exp(x^2 - y^2) - 1 = 0.$$

que está situada en el origen. Definiendo $\mathbf F(x,y) = (F_1(x,y), F_2(x,y))$ con $F_1(x,y) = \exp(x^2 + y^2) - 1$ y $F_2(x,y) = \exp(x^2 - y^2) - 1$, se calcula la siguiente matriz jacobiana:

$$
D\mathbf F(x, y) = 
\left(\begin{array}{cc} 
2x\exp(x^2 + y^2) & 2y\exp(x^2 + y^2)\\
2x\exp(x^2 - y^2) & -2y\exp(x^2 - y^2)\\
\end{array}\right).
$$

Obsérvese que la matriz $D \mathbf F$ es singular en la solución $(x,y)=(0,0)$.

El siguiente código resuelve el sistema de ecuaciones no lineales utilizando la implementación previa del método de Newton.

In [4]:
import numpy as np

# Definimos la funcion
def f(x):

    fval = np.array([np.exp(x[0]**2 + x[1]**2)-1,
                     np.exp(x[0]**2 - x[1]**2)-1])

    return fval

# Definimos de la matriz jacobiana
def df(x):

    jac = np.array([[np.exp(x[0]**2 + x[1]**2)*2*x[0],  np.exp(x[0]**2 + x[1]**2)*2*x[1]],
                    [np.exp(x[0]**2 - x[1]**2)*2*x[0], -np.exp(x[0]**2 - x[1]**2)*2*x[1]]])

    return jac

x0     = np.array([0.1, 0.1]) # Iterante inicial
tol    = 1e-8 # Tolerancia
nitmax = 40   # Numero maximo de iteraciones

x, res, nit = newton_sist(f, df, x0, tol, nitmax)

Iteracion 1, x* = [0.050496683, 0.050496683] y |x-xn| = 7.000826191e-02
Iteracion 2, x* = [0.025312613, 0.025312613] y |x-xn| = 3.561565308e-02
Iteracion 3, x* = [0.012664413, 0.012664413] y |x-xn| = 1.788725730e-02
Iteracion 4, x* = [0.006333222, 0.006333222] y |x-xn| = 8.953655842e-03
Iteracion 5, x* = [0.003166738, 0.003166738] y |x-xn| = 4.478084434e-03
Iteracion 6, x* = [0.001583385, 0.001583385] y |x-xn| = 2.239199379e-03
Iteracion 7, x* = [0.000791694, 0.000791694] y |x-xn| = 1.119619338e-03
Iteracion 8, x* = [0.000395847, 0.000395847] y |x-xn| = 5.598121249e-04
Iteracion 9, x* = [0.000197924, 0.000197924] y |x-xn| = 2.799063695e-04
Iteracion 10, x* = [0.000098962, 0.000098962] y |x-xn| = 1.399532233e-04
Iteracion 11, x* = [0.000049481, 0.000049481] y |x-xn| = 6.997661623e-05
Iteracion 12, x* = [0.000024740, 0.000024740] y |x-xn| = 3.498830865e-05
Iteracion 13, x* = [0.000012370, 0.000012370] y |x-xn| = 1.749415586e-05
Iteracion 14, x* = [0.000006185, 0.000006185] y |x-xn| = 8.7

Como se ve, la velocidad de convergencia del método ya no es en este caso cuadrática sino lineal. Se repite así la misma situación que en el caso escalar (una ecuación con una incógnita).   

## EJERCICIOS

**Ejercicio 1.** Utilizando el método de Newton para sistemas, resolver el problema descrito en la diapositiva 29 (intersección de dos circunferencias). Comprobar el resultado usando el mismo valor inicial de $x_0$. Cambiando el valor inicial, obtener ambas soluciones. 

AVANZADO: utilizando distintas condiciones iniciales, replicar el gráfico de la diapositiva 31. 

**Ejercicio 2.** Utilizando el método de Newton para sistemas, resolver el problema descrito en la diapositiva 37. Probar distintas condiciones iniciales para comprobar el efecto de las cuencas de atracción. 

AVANZADO: utilizando distintas condiciones iniciales, replicar el gráfico de la diapositiva 37. 

**Ejercicio 3.** AVANZADO: Implementar método de Newton amortiguado (slides 47-50). Utilizando el método de Newton para sistemas amortiguado, resolver el problema descrito en la diapositiva 37. Probar distintas condiciones iniciales para comprobar el efecto de las cuencas de atracción. Utilizando distintas condiciones iniciales y distintos valores de la constante de amortiguación, replicar el gráfico de la diapositiva 51.